In [ ]:
# chunked processing with ERA5 & MERRA2 aerosols with units, corrected for accurate wind calculations. 

In [5]:
import os
import math
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

# ============================
#  CONFIGURATION
# ============================
# ERA5 data paths
era5_base_path = r"Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats"
# AP (MERRA2 aerosol) data path
ap_base_path = r"Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats"

# Output directory for the aggregated results
output_dir = r"Z:\Thesis\Data\Met\ERA5_parquet_test\stats_output"
os.makedirs(output_dir, exist_ok=True)

# Years to process: 1980-2000 and 2013-2023
years = list(range(1980, 2001)) + list(range(2013, 2024))

# List of countries for ERA5 processing (for surface/pressure)
countries = [
    # "Bahrain",
    "Saudi_Arabia",
    "Oman",
    "Qatar",
    "United_Arab_Emirates",
    "Kuwait",
    "Yemen"
]

# ============================
#  DICTIONARIES: ERA5 DATA
# ============================
# File naming conventions for ERA5 surface and pressure data
surface_file_naming = {
    "2m_temperature": "2m_temperature",
    "2m_dewpoint_temperature": "2m_dewpoint_temperature",
    "10m_u_component_of_wind": "10m_u_component_of_wind",
    "10m_v_component_of_wind": "10m_v_component_of_wind",
    "surface_solar_radiation_downwards": "surface_solar_radiation_downwards",
    "surface_thermal_radiation_downwards": "surface_thermal_radiation_downwards",
    "evaporation": "evaporation",
    "potential_evaporation": "potential_evaporation",
    "sea_surface_temperature": "sea_surface_temperature",
    "soil_temperature_level_1": "soil_temperature_level_1",
    "soil_temperature_level_2": "soil_temperature_level_2",
    "soil_temperature_level_3": "soil_temperature_level_3",
    "soil_temperature_level_4": "soil_temperature_level_4",
    "volumetric_soil_water_layer_1": "volumetric_soil_water_layer_1",
    "volumetric_soil_water_layer_2": "volumetric_soil_water_layer_2",
    "volumetric_soil_water_layer_3": "volumetric_soil_water_layer_3",
    "volumetric_soil_water_layer_4": "volumetric_soil_water_layer_4",
    "leaf_area_index_high_vegetation": "leaf_area_index_high_vegetation",
    "leaf_area_index_low_vegetation": "leaf_area_index_low_vegetation",
    "surface_pressure": "surface_pressure",
    "mean_sea_level_pressure": "mean_sea_level_pressure",
    "convective_available_potential_energy": "convective_available_potential_energy",
    "geopotential": "geopotential",
    "instantaneous_10m_wind_gust": "instantaneous_10m_wind_gust",
    "total_precipitation": "total_precipitation",
    "k_index": "k_index",
}

pressure_file_naming = {
    "geopotential": "geopotential",
    "relative_humidity": "relative_humidity",
    "temperature": "temperature",
    "u_component_of_wind": "u_component_of_wind",
    "v_component_of_wind": "v_component_of_wind",
    "vertical_velocity": "vertical_velocity",
    "vorticity": "vorticity",
}

# Variable abbreviations (assumed file columns are named as "{abbr}_mean")
variable_abbreviations = {
    "2m_temperature": "t2m",
    "2m_dewpoint_temperature": "d2m",
    "10m_u_component_of_wind": "u10",
    "10m_v_component_of_wind": "v10",
    "surface_solar_radiation_downwards": "ssrd",
    "surface_thermal_radiation_downwards": "strd",
    "evaporation": "e",
    "potential_evaporation": "pev",
    "sea_surface_temperature": "sst",
    "soil_temperature_level_1": "stl1",
    "soil_temperature_level_2": "stl2",
    "soil_temperature_level_3": "stl3",
    "soil_temperature_level_4": "stl4",
    "volumetric_soil_water_layer_1": "swvl1",
    "volumetric_soil_water_layer_2": "swvl2",
    "volumetric_soil_water_layer_3": "swvl3",
    "volumetric_soil_water_layer_4": "swvl4",
    "leaf_area_index_high_vegetation": "lai_hv",
    "leaf_area_index_low_vegetation": "lai_lv",
    "surface_pressure": "sp",
    "mean_sea_level_pressure": "msl",
    "convective_available_potential_energy": "cape",
    "geopotential": "z",
    "instantaneous_10m_wind_gust": "i10fg",
    "total_precipitation": "tp",
    "k_index": "kx",
    "relative_humidity": "r",
    "temperature": "t",
    "u_component_of_wind": "u",
    "v_component_of_wind": "v",
    "vertical_velocity": "w",
    "vorticity": "vo",
}

# Build expected column names for ERA5 surface and pressure separately
surface_vars = list(surface_file_naming.keys())
pressure_vars = list(pressure_file_naming.keys())

surface_columns = [f"{variable_abbreviations[var]}_mean" for var in surface_vars if var in variable_abbreviations]
pressure_columns = [f"{variable_abbreviations[var]}_mean" for var in pressure_vars if var in variable_abbreviations]

# ============================
#  PARAMETER UNITS: ERA5 DATA
# ============================
param_units = {
    "t2m_mean":        "K",
    "d2m_mean":        "K",
    "sst_mean":        "K",
    "stl1_mean":       "K",
    "stl2_mean":       "K",
    "stl3_mean":       "K",
    "stl4_mean":       "K",
    "u10_mean":        "m/s",
    "v10_mean":        "m/s",
    "ssrd_mean":       "J/m^2",
    "strd_mean":       "J/m^2",
    "e_mean":          "m of water",
    "pev_mean":        "m of water",
    "swvl1_mean":      "m^3/m^3",
    "swvl2_mean":      "m^3/m^3",
    "swvl3_mean":      "m^3/m^3",
    "swvl4_mean":      "m^3/m^3",
    "lai_hv_mean":     "m^2/m^2",
    "lai_lv_mean":     "m^2/m^2",
    "sp_mean":         "Pa",
    "msl_mean":        "Pa",
    "cape_mean":       "J/kg",
    "z_mean":          "m",
    "i10fg_mean":      "m/s",
    "tp_mean":         "m of water",
    "kx_mean":         "K",
    "r_mean":          "",     # Relative humidity
    "t_mean":          "K",    # Pressure temperature
    "u_mean":          "m/s",
    "v_mean":          "m/s",
    "w_mean":          "m/s",
    "vo_mean":         "1/s",
}

# Temperature variables to convert (from Kelvin to Celsius)
surface_temp_vars = ["t2m_mean", "d2m_mean", "sst_mean", "stl1_mean", "stl2_mean", "stl3_mean", "stl4_mean"]
pressure_temp_vars = ["t_mean"]

# ============================
#  INITIALIZE AGGREGATORS: ERA5 DATA
# ============================
# Generic aggregators for numeric columns
surface_sum = {}
surface_count = {}
pressure_sum = {}
pressure_count = {}

def update_aggregates(df_chunk, expected_cols, sum_dict, count_dict):
    """Update sum and count for each expected column in the chunk."""
    for col in df_chunk.columns:
        if col in expected_cols:
            s = df_chunk[col].sum()
            n = df_chunk[col].count()
            sum_dict[col] = sum_dict.get(col, 0) + s
            count_dict[col] = count_dict.get(col, 0) + n

# Additional accumulators for wind (using individual observations)

# For surface wind (10-m)
surface_wind_speed_sum = 0.0
surface_wind_speed_count = 0
surface_wind_dir_sin_sum = 0.0
surface_wind_dir_cos_sum = 0.0
surface_wind_dir_count = 0

# For pressure wind
pressure_wind_speed_sum = 0.0
pressure_wind_speed_count = 0
pressure_wind_dir_sin_sum = 0.0
pressure_wind_dir_cos_sum = 0.0
pressure_wind_dir_count = 0

# ============================
#  PROCESS ERA5 FILES (SURFACE & PRESSURE)
# ============================
for yr in years:
    era5_year_path = os.path.join(era5_base_path, str(yr))
    if not os.path.isdir(era5_year_path):
        print(f"Year folder not found: {era5_year_path}")
        continue

    for country in countries:
        # Construct file paths for surface and pressure files
        surface_file = os.path.join(era5_year_path, f"{country}_{yr}_surface_monthly_stats.parquet")
        pressure_file = os.path.join(era5_year_path, f"{country}_{yr}_pressure_monthly_stats.parquet")
        
        # Process surface file
        if os.path.exists(surface_file):
            try:
                pf = pq.ParquetFile(surface_file)
                file_columns = set(pf.schema.names)
                cols_to_read = list(set(surface_columns).intersection(file_columns))
                if not cols_to_read:
                    print(f"No matching surface columns in {surface_file}")
                else:
                    for batch in pf.iter_batches(columns=cols_to_read):
                        df_chunk = batch.to_pandas()
                        update_aggregates(df_chunk, surface_columns, surface_sum, surface_count)
                        # Wind calculations for each observation using u10 and v10
                        if "u10_mean" in df_chunk.columns and "v10_mean" in df_chunk.columns:
                            speeds = np.sqrt(df_chunk["u10_mean"]**2 + df_chunk["v10_mean"]**2)
                            surface_wind_speed_sum += speeds.sum()
                            surface_wind_speed_count += speeds.count()
                            theta = np.degrees(np.arctan2(df_chunk["v10_mean"], df_chunk["u10_mean"]))
                            directions = (270.0 - theta) % 360.0
                            rad = np.radians(directions)
                            surface_wind_dir_sin_sum += np.sin(rad).sum()
                            surface_wind_dir_cos_sum += np.cos(rad).sum()
                            surface_wind_dir_count += len(rad)
            except Exception as e:
                print(f"Error processing {surface_file}: {e}")
        else:
            print(f"Surface file not found: {surface_file}")

        # Process pressure file
        if os.path.exists(pressure_file):
            try:
                pf = pq.ParquetFile(pressure_file)
                file_columns = set(pf.schema.names)
                cols_to_read = list(set(pressure_columns).intersection(file_columns))
                if not cols_to_read:
                    print(f"No matching pressure columns in {pressure_file}")
                else:
                    for batch in pf.iter_batches(columns=cols_to_read):
                        df_chunk = batch.to_pandas()
                        update_aggregates(df_chunk, pressure_columns, pressure_sum, pressure_count)
                        if "u_mean" in df_chunk.columns and "v_mean" in df_chunk.columns:
                            speeds = np.sqrt(df_chunk["u_mean"]**2 + df_chunk["v_mean"]**2)
                            pressure_wind_speed_sum += speeds.sum()
                            pressure_wind_speed_count += speeds.count()
                            theta = np.degrees(np.arctan2(df_chunk["v_mean"], df_chunk["u_mean"]))
                            directions = (270.0 - theta) % 360.0
                            rad = np.radians(directions)
                            pressure_wind_dir_sin_sum += np.sin(rad).sum()
                            pressure_wind_dir_cos_sum += np.cos(rad).sum()
                            pressure_wind_dir_count += len(rad)
            except Exception as e:
                print(f"Error processing {pressure_file}: {e}")
        else:
            print(f"Pressure file not found: {pressure_file}")

def compute_means(sum_dict, count_dict):
    means = {}
    for col, total in sum_dict.items():
        cnt = count_dict.get(col, 0)
        means[col] = total / cnt if cnt > 0 else None
    return means

surface_means = compute_means(surface_sum, surface_count)
pressure_means = compute_means(pressure_sum, pressure_count)

# ---------------------------
# Temperature conversion for ERA5 data
# ---------------------------
for tvar in surface_temp_vars:
    if tvar in surface_means and surface_means[tvar] is not None:
        surface_means[tvar] -= 273.15
        param_units[tvar] = "°C"
for tvar in pressure_temp_vars:
    if tvar in pressure_means and pressure_means[tvar] is not None:
        pressure_means[tvar] -= 273.15
        param_units[tvar] = "°C"

# ---------------------------
# Wind calculations for ERA5 data
# ---------------------------
if surface_wind_speed_count > 0:
    avg_surface_wind_speed = surface_wind_speed_sum / surface_wind_speed_count
    avg_sin = surface_wind_dir_sin_sum / surface_wind_dir_count
    avg_cos = surface_wind_dir_cos_sum / surface_wind_dir_count
    avg_surface_wind_dir = math.degrees(math.atan2(avg_sin, avg_cos)) % 360
    surface_means["wind_speed_10m"] = avg_surface_wind_speed
    surface_means["wind_dir_10m"] = avg_surface_wind_dir
    param_units["wind_speed_10m"] = "m/s"
    param_units["wind_dir_10m"] = "deg_from_N"
else:
    print("No surface wind observations to compute wind speed/direction.")

if pressure_wind_speed_count > 0:
    avg_pressure_wind_speed = pressure_wind_speed_sum / pressure_wind_speed_count
    avg_sin = pressure_wind_dir_sin_sum / pressure_wind_dir_count
    avg_cos = pressure_wind_dir_cos_sum / pressure_wind_dir_count
    avg_pressure_wind_dir = math.degrees(math.atan2(avg_sin, avg_cos)) % 360
    pressure_means["wind_speed_pressure"] = avg_pressure_wind_speed
    pressure_means["wind_dir_pressure"] = avg_pressure_wind_dir
    param_units["wind_speed_pressure"] = "m/s"
    param_units["wind_dir_pressure"] = "deg_from_N"
else:
    print("No pressure wind observations to compute wind speed/direction.")

# ============================
#  PROCESS MERRA2 AP DATA
# ============================
# AP (MERRA2 aerosol) variables and their expected units.
ap_vars = [
    #"DUANGSTR", 
    "DUCMASS25_mean", 
    "DUCMASS_mean", 
    #"DUEXTT25", 
    #"DUEXTTAU",
    "DUFLUXU_mean", 
    "DUFLUXV_mean", 
    #"DUSCAT25", 
    #"DUSCATAU", 
    "DUSMASS25_mean", 
    "DUSMASS_mean"
]
# Define AP units (adjust units as needed)
ap_param_units = {
    #"DUANGSTR": "",
    "DUCMASS25_mean": "kg/m^2",
    "DUCMASS_mean": "kg/m^2",
    #"DUEXTT25": "",
    #"DUEXTTAU": "",
    "DUFLUXU_mean": "kg/(m/s)",
    "DUFLUXV_mean": "kg/(m/s)",
    #"DUSCAT25": "",
    #"DUSCATAU": "",
    "DUSMASS25_mean": "kg/m^3",
    "DUSMASS_mean": "kg/m^3",
}
# We will also derive wind speed and direction from DUFLUXU/DUFLUXV.
# Append these keys with their units:
ap_param_units["wind_speed_duflux"] = "kg/(m/s)"
ap_param_units["wind_dir_duflux"] = "deg_from_N"

# Initialize aggregators for AP data
ap_sum = {}
ap_count = {}

# Wind accumulators for AP DUFLUX variables
ap_wind_speed_sum = 0.0
ap_wind_speed_count = 0
ap_wind_dir_sin_sum = 0.0
ap_wind_dir_cos_sum = 0.0
ap_wind_dir_count = 0

def update_ap_aggregates(df_chunk, expected_cols, sum_dict, count_dict):
    """Update sum and count for AP expected columns."""
    for col in df_chunk.columns:
        if col in expected_cols:
            s = df_chunk[col].sum()
            n = df_chunk[col].count()
            sum_dict[col] = sum_dict.get(col, 0) + s
            count_dict[col] = count_dict.get(col, 0) + n

# Process AP files (one file per year, no countries)
for yr in years:
    ap_year_path = os.path.join(ap_base_path, str(yr))
    # Construct the AP file path; e.g., "MERRA2_1980_monthly_stats.parquet"
    ap_file = os.path.join(ap_year_path, f"MERRA2_{yr}_monthly_stats.parquet")
    if os.path.exists(ap_file):
        try:
            pf = pq.ParquetFile(ap_file)
            file_columns = set(pf.schema.names)
            cols_to_read = list(set(ap_vars).intersection(file_columns))
            if not cols_to_read:
                print(f"No matching AP columns in {ap_file}")
            else:
                for batch in pf.iter_batches(columns=cols_to_read):
                    df_chunk = batch.to_pandas()
                    update_ap_aggregates(df_chunk, ap_vars, ap_sum, ap_count)
                    # Wind: compute per-row wind speed/direction from DUFLUXU and DUFLUXV
                    if "DUFLUXU_mean" in df_chunk.columns and "DUFLUXV_mean" in df_chunk.columns:
                        speeds = np.sqrt(df_chunk["DUFLUXU_mean"]**2 + df_chunk["DUFLUXV_mean"]**2)
                        ap_wind_speed_sum += speeds.sum()
                        ap_wind_speed_count += speeds.count()
                        theta = np.degrees(np.arctan2(df_chunk["DUFLUXV_mean"], df_chunk["DUFLUXU_mean"]))
                        directions = (270.0 - theta) % 360.0
                        rad = np.radians(directions)
                        ap_wind_dir_sin_sum += np.sin(rad).sum()
                        ap_wind_dir_cos_sum += np.cos(rad).sum()
                        ap_wind_dir_count += len(rad)
        except Exception as e:
            print(f"Error processing {ap_file}: {e}")
    else:
        print(f"AP file not found: {ap_file}")

ap_means = compute_means(ap_sum, ap_count)

# Compute AP wind from DUFLUXU/DUFLUXV if available
if ap_wind_speed_count > 0:
    avg_ap_wind_speed = ap_wind_speed_sum / ap_wind_speed_count
    avg_sin = ap_wind_dir_sin_sum / ap_wind_dir_count
    avg_cos = ap_wind_dir_cos_sum / ap_wind_dir_count
    avg_ap_wind_dir = math.degrees(math.atan2(avg_sin, avg_cos)) % 360
    ap_means["wind_speed_duflux"] = avg_ap_wind_speed
    ap_means["wind_dir_duflux"] = avg_ap_wind_dir
else:
    print("No AP MERRA-2 wind observations to compute wind speed/direction.")

# ============================
#  SAVE RESULTS
# ============================
def save_results(means_dict, units_dict, filename):
    results = []
    for param, mean_val in means_dict.items():
        unit = units_dict.get(param, "")
        results.append({"Parameter": param, "Mean_Value": mean_val, "Units": unit})
    df = pd.DataFrame(results)
    out_path = os.path.join(output_dir, filename)
    df.to_csv(out_path, index=False)
    print(f"Results saved to {out_path}")

# Save ERA5 surface and pressure results
save_results(surface_means, param_units, "ERA5_surface_means_with_wind_calcs_chunked.csv")
save_results(pressure_means, param_units, "ERA5_pressure_means_with_wind_calcs_chunked.csv")
# Save AP (MERRA2 aerosol) results using AP units
save_results(ap_means, ap_param_units, "MERRA2_AP_means_with_wind_calcs_chunked.csv")


Results saved to Z:\Thesis\Data\Met\ERA5_parquet_test\stats_output\ERA5_surface_means_with_wind_calcs_chunked.csv
Results saved to Z:\Thesis\Data\Met\ERA5_parquet_test\stats_output\ERA5_pressure_means_with_wind_calcs_chunked.csv
Results saved to Z:\Thesis\Data\Met\ERA5_parquet_test\stats_output\MERRA2_AP_means_with_wind_calcs_chunked.csv


In [7]:
#analysis for the lowest 5000 feet of the atmosphere (level = 1000, 925, 850mb)

import os
import math
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

# ============================
#  CONFIGURATION
# ============================
# ERA5 data paths
era5_base_path = r"Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats"
# AP (MERRA2 aerosol) data path
ap_base_path = r"Z:\Thesis\Data\GEE\MERRA2_aer\MERRA2_num_data\Monthly_stats"

# Output directory for the aggregated results
output_dir = r"Z:\Thesis\Data\Met\ERA5_parquet_test\stats_output"
os.makedirs(output_dir, exist_ok=True)

# Years to process: 1980-2000 and 2013-2023
years = list(range(1980, 2001)) + list(range(2013, 2024))

# List of countries for ERA5 processing (for surface/pressure)
countries = [
    # "Bahrain",
    "Saudi_Arabia",
    "Oman",
    "Qatar",
    "United_Arab_Emirates",
    "Kuwait",
    "Yemen"
]

# ============================
#  DICTIONARIES: ERA5 DATA
# ============================
# File naming conventions for ERA5 surface and pressure data
surface_file_naming = {
    "2m_temperature": "2m_temperature",
    "2m_dewpoint_temperature": "2m_dewpoint_temperature",
    "10m_u_component_of_wind": "10m_u_component_of_wind",
    "10m_v_component_of_wind": "10m_v_component_of_wind",
    "surface_solar_radiation_downwards": "surface_solar_radiation_downwards",
    "surface_thermal_radiation_downwards": "surface_thermal_radiation_downwards",
    "evaporation": "evaporation",
    "potential_evaporation": "potential_evaporation",
    "sea_surface_temperature": "sea_surface_temperature",
    "soil_temperature_level_1": "soil_temperature_level_1",
    "soil_temperature_level_2": "soil_temperature_level_2",
    "soil_temperature_level_3": "soil_temperature_level_3",
    "soil_temperature_level_4": "soil_temperature_level_4",
    "volumetric_soil_water_layer_1": "volumetric_soil_water_layer_1",
    "volumetric_soil_water_layer_2": "volumetric_soil_water_layer_2",
    "volumetric_soil_water_layer_3": "volumetric_soil_water_layer_3",
    "volumetric_soil_water_layer_4": "volumetric_soil_water_layer_4",
    "leaf_area_index_high_vegetation": "leaf_area_index_high_vegetation",
    "leaf_area_index_low_vegetation": "leaf_area_index_low_vegetation",
    "surface_pressure": "surface_pressure",
    "mean_sea_level_pressure": "mean_sea_level_pressure",
    "convective_available_potential_energy": "convective_available_potential_energy",
    "geopotential": "geopotential",
    "instantaneous_10m_wind_gust": "instantaneous_10m_wind_gust",
    "total_precipitation": "total_precipitation",
    "k_index": "k_index",
}

pressure_file_naming = {
    "geopotential": "geopotential",
    "relative_humidity": "relative_humidity",
    "temperature": "temperature",
    "u_component_of_wind": "u_component_of_wind",
    "v_component_of_wind": "v_component_of_wind",
    "vertical_velocity": "vertical_velocity",
    "vorticity": "vorticity",
}

# Variable abbreviations (assumed file columns are named as "{abbr}_mean")
variable_abbreviations = {
    "2m_temperature": "t2m",
    "2m_dewpoint_temperature": "d2m",
    "10m_u_component_of_wind": "u10",
    "10m_v_component_of_wind": "v10",
    "surface_solar_radiation_downwards": "ssrd",
    "surface_thermal_radiation_downwards": "strd",
    "evaporation": "e",
    "potential_evaporation": "pev",
    "sea_surface_temperature": "sst",
    "soil_temperature_level_1": "stl1",
    "soil_temperature_level_2": "stl2",
    "soil_temperature_level_3": "stl3",
    "soil_temperature_level_4": "stl4",
    "volumetric_soil_water_layer_1": "swvl1",
    "volumetric_soil_water_layer_2": "swvl2",
    "volumetric_soil_water_layer_3": "swvl3",
    "volumetric_soil_water_layer_4": "swvl4",
    "leaf_area_index_high_vegetation": "lai_hv",
    "leaf_area_index_low_vegetation": "lai_lv",
    "surface_pressure": "sp",
    "mean_sea_level_pressure": "msl",
    "convective_available_potential_energy": "cape",
    "geopotential": "z",
    "instantaneous_10m_wind_gust": "i10fg",
    "total_precipitation": "tp",
    "k_index": "kx",
    "relative_humidity": "r",
    "temperature": "t",
    "u_component_of_wind": "u",
    "v_component_of_wind": "v",
    "vertical_velocity": "w",
    "vorticity": "vo",
}

# Build expected column names for ERA5 surface and pressure separately
surface_vars = list(surface_file_naming.keys())
pressure_vars = list(pressure_file_naming.keys())

surface_columns = [f"{variable_abbreviations[var]}_mean" for var in surface_vars if var in variable_abbreviations]
pressure_columns = [f"{variable_abbreviations[var]}_mean" for var in pressure_vars if var in variable_abbreviations]

# ============================
#  PARAMETER UNITS: ERA5 DATA
# ============================
param_units = {
    "t2m_mean":        "K",
    "d2m_mean":        "K",
    "sst_mean":        "K",
    "stl1_mean":       "K",
    "stl2_mean":       "K",
    "stl3_mean":       "K",
    "stl4_mean":       "K",
    "u10_mean":        "m/s",
    "v10_mean":        "m/s",
    "ssrd_mean":       "J/m^2",
    "strd_mean":       "J/m^2",
    "e_mean":          "m of water",
    "pev_mean":        "m of water",
    "swvl1_mean":      "m^3/m^3",
    "swvl2_mean":      "m^3/m^3",
    "swvl3_mean":      "m^3/m^3",
    "swvl4_mean":      "m^3/m^3",
    "lai_hv_mean":     "m^2/m^2",
    "lai_lv_mean":     "m^2/m^2",
    "sp_mean":         "Pa",
    "msl_mean":        "Pa",
    "cape_mean":       "J/kg",
    "z_mean":          "m",
    "i10fg_mean":      "m/s",
    "tp_mean":         "m of water",
    "kx_mean":         "K",
    "r_mean":          "",     # Relative humidity
    "t_mean":          "K",    # Pressure temperature
    "u_mean":          "m/s",
    "v_mean":          "m/s",
    "w_mean":          "m/s",
    "vo_mean":         "1/s",
}

# Temperature variables to convert (from Kelvin to Celsius)
surface_temp_vars = ["t2m_mean", "d2m_mean", "sst_mean", "stl1_mean", "stl2_mean", "stl3_mean", "stl4_mean"]
pressure_temp_vars = ["t_mean"]

# ============================
#  INITIALIZE AGGREGATORS: ERA5 DATA
# ============================
# Generic aggregators for numeric columns
surface_sum = {}
surface_count = {}
pressure_sum = {}
pressure_count = {}

def update_aggregates(df_chunk, expected_cols, sum_dict, count_dict):
    """Update sum and count for each expected column in the chunk."""
    for col in df_chunk.columns:
        if col in expected_cols:
            s = df_chunk[col].sum()
            n = df_chunk[col].count()
            sum_dict[col] = sum_dict.get(col, 0) + s
            count_dict[col] = count_dict.get(col, 0) + n

# Additional accumulators for wind (using individual observations)

# For surface wind (10-m)
surface_wind_speed_sum = 0.0
surface_wind_speed_count = 0
surface_wind_dir_sin_sum = 0.0
surface_wind_dir_cos_sum = 0.0
surface_wind_dir_count = 0

# For pressure wind
pressure_wind_speed_sum = 0.0
pressure_wind_speed_count = 0
pressure_wind_dir_sin_sum = 0.0
pressure_wind_dir_cos_sum = 0.0
pressure_wind_dir_count = 0

# ============================
#  PROCESS ERA5 FILES (SURFACE & PRESSURE)
# ============================
for yr in years:
    era5_year_path = os.path.join(era5_base_path, str(yr))
    if not os.path.isdir(era5_year_path):
        print(f"Year folder not found: {era5_year_path}")
        continue

    for country in countries:
        # Construct file paths for surface and pressure files
        surface_file = os.path.join(era5_year_path, f"{country}_{yr}_surface_monthly_stats.parquet")
        pressure_file = os.path.join(era5_year_path, f"{country}_{yr}_pressure_monthly_stats.parquet")
        
        # Process surface file
        if os.path.exists(surface_file):
            try:
                pf = pq.ParquetFile(surface_file)
                file_columns = set(pf.schema.names)
                cols_to_read = list(set(surface_columns).intersection(file_columns))
                if not cols_to_read:
                    print(f"No matching surface columns in {surface_file}")
                else:
                    for batch in pf.iter_batches(columns=cols_to_read):
                        df_chunk = batch.to_pandas()
                        update_aggregates(df_chunk, surface_columns, surface_sum, surface_count)
                        # Wind calculations for each observation using u10 and v10
                        if "u10_mean" in df_chunk.columns and "v10_mean" in df_chunk.columns:
                            speeds = np.sqrt(df_chunk["u10_mean"]**2 + df_chunk["v10_mean"]**2)
                            surface_wind_speed_sum += speeds.sum()
                            surface_wind_speed_count += speeds.count()
                            theta = np.degrees(np.arctan2(df_chunk["v10_mean"], df_chunk["u10_mean"]))
                            directions = (270.0 - theta) % 360.0
                            rad = np.radians(directions)
                            surface_wind_dir_sin_sum += np.sin(rad).sum()
                            surface_wind_dir_cos_sum += np.cos(rad).sum()
                            surface_wind_dir_count += len(rad)
            except Exception as e:
                print(f"Error processing {surface_file}: {e}")
        else:
            print(f"Surface file not found: {surface_file}")

        # Process pressure file (with filtering on 'level')
        if os.path.exists(pressure_file):
            try:
                pf = pq.ParquetFile(pressure_file)
                file_columns = set(pf.schema.names)
                cols_to_read = list(set(pressure_columns).intersection(file_columns))
                if not cols_to_read:
                    print(f"No matching pressure columns in {pressure_file}")
                else:
                    for batch in pf.iter_batches(columns=cols_to_read):
                        df_chunk = batch.to_pandas()
                        # Filter rows by key levels for sand/dust storms
                        if "level" in df_chunk.columns:
                            df_chunk = df_chunk[df_chunk["level"].isin(['1000','925','850'])]
                        if df_chunk.empty:
                            print(f"no levels found")
                            continue
                        update_aggregates(df_chunk, pressure_columns, pressure_sum, pressure_count)
                        if "u_mean" in df_chunk.columns and "v_mean" in df_chunk.columns:
                            speeds = np.sqrt(df_chunk["u_mean"]**2 + df_chunk["v_mean"]**2)
                            pressure_wind_speed_sum += speeds.sum()
                            pressure_wind_speed_count += speeds.count()
                            theta = np.degrees(np.arctan2(df_chunk["v_mean"], df_chunk["u_mean"]))
                            directions = (270.0 - theta) % 360.0
                            rad = np.radians(directions)
                            pressure_wind_dir_sin_sum += np.sin(rad).sum()
                            pressure_wind_dir_cos_sum += np.cos(rad).sum()
                            pressure_wind_dir_count += len(rad)
            except Exception as e:
                print(f"Error processing {pressure_file}: {e}")
        else:
            print(f"Pressure file not found: {pressure_file}")

def compute_means(sum_dict, count_dict):
    means = {}
    for col, total in sum_dict.items():
        cnt = count_dict.get(col, 0)
        means[col] = total / cnt if cnt > 0 else None
    return means

surface_means = compute_means(surface_sum, surface_count)
pressure_means = compute_means(pressure_sum, pressure_count)

# ---------------------------
# Temperature conversion for ERA5 data
# ---------------------------
for tvar in surface_temp_vars:
    if tvar in surface_means and surface_means[tvar] is not None:
        surface_means[tvar] -= 273.15
        param_units[tvar] = "°C"
for tvar in pressure_temp_vars:
    if tvar in pressure_means and pressure_means[tvar] is not None:
        pressure_means[tvar] -= 273.15
        param_units[tvar] = "°C"

# ---------------------------
# Wind calculations for ERA5 data
# ---------------------------
if surface_wind_speed_count > 0:
    avg_surface_wind_speed = surface_wind_speed_sum / surface_wind_speed_count
    avg_sin = surface_wind_dir_sin_sum / surface_wind_dir_count
    avg_cos = surface_wind_dir_cos_sum / surface_wind_dir_count
    avg_surface_wind_dir = math.degrees(math.atan2(avg_sin, avg_cos)) % 360
    surface_means["wind_speed_10m"] = avg_surface_wind_speed
    surface_means["wind_dir_10m"] = avg_surface_wind_dir
    param_units["wind_speed_10m"] = "m/s"
    param_units["wind_dir_10m"] = "deg_from_N"
else:
    print("No surface wind observations to compute wind speed/direction.")

if pressure_wind_speed_count > 0:
    avg_pressure_wind_speed = pressure_wind_speed_sum / pressure_wind_speed_count
    avg_sin = pressure_wind_dir_sin_sum / pressure_wind_dir_count
    avg_cos = pressure_wind_dir_cos_sum / pressure_wind_dir_count
    avg_pressure_wind_dir = math.degrees(math.atan2(avg_sin, avg_cos)) % 360
    pressure_means["wind_speed_pressure"] = avg_pressure_wind_speed
    pressure_means["wind_dir_pressure"] = avg_pressure_wind_dir
    param_units["wind_speed_pressure"] = "m/s"
    param_units["wind_dir_pressure"] = "deg_from_N"
else:
    print("No pressure wind observations to compute wind speed/direction.")

# ============================
#  PROCESS MERRA2 AP DATA
# ============================
# AP (MERRA2 aerosol) variables and their expected units.
ap_vars = [
    #"DUANGSTR", 
    "DUCMASS25_mean", 
    "DUCMASS_mean", 
    #"DUEXTT25", 
    #"DUEXTTAU",
    "DUFLUXU_mean", 
    "DUFLUXV_mean", 
    #"DUSCAT25", 
    #"DUSCATAU", 
    "DUSMASS25_mean", 
    "DUSMASS_mean"
]
# Define AP units (adjust units as needed)
ap_param_units = {
    #"DUANGSTR": "",
    "DUCMASS25_mean": "kg/m^2",
    "DUCMASS_mean": "kg/m^2",
    #"DUEXTT25": "",
    #"DUEXTTAU": "",
    "DUFLUXU_mean": "kg/(m/s)",
    "DUFLUXV_mean": "kg/(m/s)",
    #"DUSCAT25": "",
    #"DUSCATAU": "",
    "DUSMASS25_mean": "kg/m^3",
    "DUSMASS_mean": "kg/m^3",
}
# We will also derive wind speed and direction from DUFLUXU/DUFLUXV.
# Append these keys with their units:
ap_param_units["wind_speed_duflux"] = "kg/(m/s)"
ap_param_units["wind_dir_duflux"] = "deg_from_N"

# Initialize aggregators for AP data
ap_sum = {}
ap_count = {}

# Wind accumulators for AP DUFLUX variables
ap_wind_speed_sum = 0.0
ap_wind_speed_count = 0
ap_wind_dir_sin_sum = 0.0
ap_wind_dir_cos_sum = 0.0
ap_wind_dir_count = 0

def update_ap_aggregates(df_chunk, expected_cols, sum_dict, count_dict):
    """Update sum and count for AP expected columns."""
    for col in df_chunk.columns:
        if col in expected_cols:
            s = df_chunk[col].sum()
            n = df_chunk[col].count()
            sum_dict[col] = sum_dict.get(col, 0) + s
            count_dict[col] = count_dict.get(col, 0) + n

# Process AP files (one file per year, no countries)
for yr in years:
    ap_year_path = os.path.join(ap_base_path, str(yr))
    # Construct the AP file path; e.g., "MERRA2_1980_monthly_stats.parquet"
    ap_file = os.path.join(ap_year_path, f"MERRA2_{yr}_monthly_stats.parquet")
    if os.path.exists(ap_file):
        try:
            pf = pq.ParquetFile(ap_file)
            file_columns = set(pf.schema.names)
            cols_to_read = list(set(ap_vars).intersection(file_columns))
            if not cols_to_read:
                print(f"No matching AP columns in {ap_file}")
            else:
                for batch in pf.iter_batches(columns=cols_to_read):
                    df_chunk = batch.to_pandas()
                    update_ap_aggregates(df_chunk, ap_vars, ap_sum, ap_count)
                    # Wind: compute per-row wind speed/direction from DUFLUXU and DUFLUXV
                    if "DUFLUXU_mean" in df_chunk.columns and "DUFLUXV_mean" in df_chunk.columns:
                        speeds = np.sqrt(df_chunk["DUFLUXU_mean"]**2 + df_chunk["DUFLUXV_mean"]**2)
                        ap_wind_speed_sum += speeds.sum()
                        ap_wind_speed_count += speeds.count()
                        theta = np.degrees(np.arctan2(df_chunk["DUFLUXV_mean"], df_chunk["DUFLUXU_mean"]))
                        directions = (270.0 - theta) % 360.0
                        rad = np.radians(directions)
                        ap_wind_dir_sin_sum += np.sin(rad).sum()
                        ap_wind_dir_cos_sum += np.cos(rad).sum()
                        ap_wind_dir_count += len(rad)
        except Exception as e:
            print(f"Error processing {ap_file}: {e}")
    else:
        print(f"AP file not found: {ap_file}")

ap_means = compute_means(ap_sum, ap_count)

# Compute AP wind from DUFLUXU/DUFLUXV if available
if ap_wind_speed_count > 0:
    avg_ap_wind_speed = ap_wind_speed_sum / ap_wind_speed_count
    avg_sin = ap_wind_dir_sin_sum / ap_wind_dir_count
    avg_cos = ap_wind_dir_cos_sum / ap_wind_dir_count
    avg_ap_wind_dir = math.degrees(math.atan2(avg_sin, avg_cos)) % 360
    ap_means["wind_speed_duflux"] = avg_ap_wind_speed
    ap_means["wind_dir_duflux"] = avg_ap_wind_dir
else:
    print("No AP MERRA-2 wind observations to compute wind speed/direction.")

# ============================
#  SAVE RESULTS
# ============================
def save_results(means_dict, units_dict, filename):
    results = []
    for param, mean_val in means_dict.items():
        unit = units_dict.get(param, "")
        results.append({"Parameter": param, "Mean_Value": mean_val, "Units": unit})
    df = pd.DataFrame(results)
    out_path = os.path.join(output_dir, filename)
    df.to_csv(out_path, index=False)
    print(f"Results saved to {out_path}")

# Save ERA5 surface and pressure results
save_results(surface_means, param_units, "ERA5_surface_means_with_wind_calcs_chunked_.csv")
save_results(pressure_means, param_units, "ERA5_pressure_means_with_wind_calcs_chunked_below5kFT.csv")
# Save AP (MERRA2 aerosol) results using AP units
save_results(ap_means, ap_param_units, "MERRA2_AP_means_with_wind_calcs_chunked_.csv")


Results saved to Z:\Thesis\Data\Met\ERA5_parquet_test\stats_output\ERA5_surface_means_with_wind_calcs_chunked_.csv
Results saved to Z:\Thesis\Data\Met\ERA5_parquet_test\stats_output\ERA5_pressure_means_with_wind_calcs_chunked_below5kFT.csv
Results saved to Z:\Thesis\Data\Met\ERA5_parquet_test\stats_output\MERRA2_AP_means_with_wind_calcs_chunked_.csv


In [ ]:
## chunked processing, no merra2 data

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

# ------------------------- CONFIGURATION -------------------------
base_path = r"Z:\Thesis\Data\Met\ERA5_parquet_test\Monthly_Stats"
output_dir = r"Z:\Thesis\Data\Met\ERA5_parquet_test\stats_output"
os.makedirs(output_dir, exist_ok=True)

# Years to process: 1980-2000 and 2013-2024 (i.e. through 2023)
years = list(range(1980, 2001)) + list(range(2013, 2024))

# List of countries to process (Bahrain is commented out)
countries = [
    # "Bahrain",
    "Saudi_Arabia",
    "Oman",
    "Qatar",
    "United_Arab_Emirates",
    "Kuwait",
    "Yemen"
]

# ------------------------- DICTIONARIES -------------------------
surface_file_naming = {
    "2m_temperature": "2m_temperature",
    "2m_dewpoint_temperature": "2m_dewpoint_temperature",
    "10m_u_component_of_wind": "10m_u_component_of_wind",
    "10m_v_component_of_wind": "10m_v_component_of_wind",
    "surface_solar_radiation_downwards": "surface_solar_radiation_downwards",
    "surface_thermal_radiation_downwards": "surface_thermal_radiation_downwards",
    "evaporation": "evaporation",
    "potential_evaporation": "potential_evaporation",
    "sea_surface_temperature": "sea_surface_temperature",
    "soil_temperature_level_1": "soil_temperature_level_1",
    "soil_temperature_level_2": "soil_temperature_level_2",
    "soil_temperature_level_3": "soil_temperature_level_3",
    "soil_temperature_level_4": "soil_temperature_level_4",
    "volumetric_soil_water_layer_1": "volumetric_soil_water_layer_1",
    "volumetric_soil_water_layer_2": "volumetric_soil_water_layer_2",
    "volumetric_soil_water_layer_3": "volumetric_soil_water_layer_3",
    "volumetric_soil_water_layer_4": "volumetric_soil_water_layer_4",
    "leaf_area_index_high_vegetation": "leaf_area_index_high_vegetation",
    "leaf_area_index_low_vegetation": "leaf_area_index_low_vegetation",
    "surface_pressure": "surface_pressure",
    "mean_sea_level_pressure": "mean_sea_level_pressure",
    "convective_available_potential_energy": "convective_available_potential_energy",
    "geopotential": "geopotential",
    "instantaneous_10m_wind_gust": "instantaneous_10m_wind_gust",
    "total_precipitation": "total_precipitation",
    "k_index": "k_index",
}

pressure_file_naming = {
    "geopotential": "geopotential",
    "relative_humidity": "relative_humidity",
    "temperature": "temperature",
    "u_component_of_wind": "u_component_of_wind",
    "v_component_of_wind": "v_component_of_wind",
    "vertical_velocity": "vertical_velocity",
    "vorticity": "vorticity",
}

# Variable abbreviations (assumed file columns are named as "{abbr}_mean")
variable_abbreviations = {
    "2m_temperature": "t2m",
    "2m_dewpoint_temperature": "d2m",
    "10m_u_component_of_wind": "u10",
    "10m_v_component_of_wind": "v10",
    "surface_solar_radiation_downwards": "ssrd",
    "surface_thermal_radiation_downwards": "strd",
    "evaporation": "e",
    "potential_evaporation": "pev",
    "sea_surface_temperature": "sst",
    "soil_temperature_level_1": "stl1",
    "soil_temperature_level_2": "stl2",
    "soil_temperature_level_3": "stl3",
    "soil_temperature_level_4": "stl4",
    "volumetric_soil_water_layer_1": "swvl1",
    "volumetric_soil_water_layer_2": "swvl2",
    "volumetric_soil_water_layer_3": "swvl3",
    "volumetric_soil_water_layer_4": "swvl4",
    "leaf_area_index_high_vegetation": "lai_hv",
    "leaf_area_index_low_vegetation": "lai_lv",
    "surface_pressure": "sp",
    "mean_sea_level_pressure": "msl",
    "convective_available_potential_energy": "cape",
    "geopotential": "z",
    "instantaneous_10m_wind_gust": "i10fg",
    "total_precipitation": "tp",
    "k_index": "kx",
    "relative_humidity": "r",
    "temperature": "t",
    "u_component_of_wind": "u",
    "v_component_of_wind": "v",
    "vertical_velocity": "w",
    "vorticity": "vo",
}

# ------------------------- EXPECTED COLUMNS -------------------------
# Build expected column names for each file type
surface_vars = list(surface_file_naming.keys())
pressure_vars = list(pressure_file_naming.keys())

surface_columns = [f"{variable_abbreviations[var]}_mean" for var in surface_vars if var in variable_abbreviations]
pressure_columns = [f"{variable_abbreviations[var]}_mean" for var in pressure_vars if var in variable_abbreviations]

# ------------------------- PARAMETER UNITS -------------------------
param_units = {
    "t2m_mean":        "K",
    "d2m_mean":        "K",
    "sst_mean":        "K",
    "stl1_mean":       "K",
    "stl2_mean":       "K",
    "stl3_mean":       "K",
    "stl4_mean":       "K",
    "u10_mean":        "m/s",
    "v10_mean":        "m/s",
    "ssrd_mean":       "J/m^2",
    "strd_mean":       "J/m^2",
    "e_mean":          "m of water",
    "pev_mean":        "m of water",
    "swvl1_mean":      "m^3/m^3",
    "swvl2_mean":      "m^3/m^3",
    "swvl3_mean":      "m^3/m^3",
    "swvl4_mean":      "m^3/m^3",
    "lai_hv_mean":     "m^2/m^2",
    "lai_lv_mean":     "m^2/m^2",
    "sp_mean":         "Pa",
    "msl_mean":        "Pa",
    "cape_mean":       "J/kg",
    "z_mean":          "m",
    "i10fg_mean":      "m/s",
    "tp_mean":         "m of water",
    "kx_mean":         "K",
    "r_mean":          "",     # Relative humidity
    "t_mean":          "K",    # Pressure temperature
    "u_mean":          "m/s",
    "v_mean":          "m/s",
    "w_mean":          "m/s",
    "vo_mean":         "1/s",
    "DUSMASS_mean":    "kg/m^3",
    "DUSMASS25_mean":  "kg/m^3",
    "DUFLUXU_mean":    "kg/(m·s)",
    "DUFLUXV_mean":    "kg/(m·s)",
}

# Temperature variables to convert (Kelvin -> Celsius)
surface_temp_vars = ["t2m_mean", "d2m_mean", "sst_mean", "stl1_mean", "stl2_mean", "stl3_mean", "stl4_mean"]
pressure_temp_vars = ["t_mean"]

# ------------------------- INITIALIZE AGGREGATORS -------------------------
# Generic aggregators for numeric columns
surface_sum = {}
surface_count = {}
pressure_sum = {}
pressure_count = {}

def update_aggregates(df_chunk, expected_cols, sum_dict, count_dict):
    """Update sum and count for each expected column in the chunk."""
    for col in df_chunk.columns:
        if col in expected_cols:
            s = df_chunk[col].sum()
            n = df_chunk[col].count()
            sum_dict[col] = sum_dict.get(col, 0) + s
            count_dict[col] = count_dict.get(col, 0) + n

# Additional accumulators for wind speed and wind direction using individual observations

# For surface wind (10-m)
surface_wind_speed_sum = 0.0
surface_wind_speed_count = 0
surface_wind_dir_sin_sum = 0.0
surface_wind_dir_cos_sum = 0.0
surface_wind_dir_count = 0

# For pressure wind
pressure_wind_speed_sum = 0.0
pressure_wind_speed_count = 0
pressure_wind_dir_sin_sum = 0.0
pressure_wind_dir_cos_sum = 0.0
pressure_wind_dir_count = 0

# ------------------------- PROCESS FILES IN CHUNKS -------------------------
for yr in years:
    year_path = os.path.join(base_path, str(yr))
    if not os.path.isdir(year_path):
        print(f"Year folder not found: {year_path}")
        continue

    for country in countries:
        # Construct file paths
        surface_file = os.path.join(year_path, f"{country}_{yr}_surface_monthly_stats.parquet")
        pressure_file = os.path.join(year_path, f"{country}_{yr}_pressure_monthly_stats.parquet")
        
        # Process surface file
        if os.path.exists(surface_file):
            try:
                pf = pq.ParquetFile(surface_file)
                file_columns = set(pf.schema.names)
                cols_to_read = list(set(surface_columns).intersection(file_columns))
                if not cols_to_read:
                    print(f"No matching surface columns in {surface_file}")
                else:
                    for batch in pf.iter_batches(columns=cols_to_read):
                        df_chunk = batch.to_pandas()
                        # Update generic aggregates
                        update_aggregates(df_chunk, surface_columns, surface_sum, surface_count)
                        # Compute wind speed and wind direction per observation if available
                        if "u10_mean" in df_chunk.columns and "v10_mean" in df_chunk.columns:
                            # Compute individual wind speeds
                            speeds = np.sqrt(df_chunk["u10_mean"]**2 + df_chunk["v10_mean"]**2)
                            surface_wind_speed_sum += speeds.sum()
                            surface_wind_speed_count += speeds.count()
                            # Compute individual wind directions (meteorological "from" direction)
                            theta = np.degrees(np.arctan2(df_chunk["v10_mean"], df_chunk["u10_mean"]))
                            directions = (270.0 - theta) % 360.0
                            rad = np.radians(directions)
                            surface_wind_dir_sin_sum += np.sin(rad).sum()
                            surface_wind_dir_cos_sum += np.cos(rad).sum()
                            surface_wind_dir_count += len(rad)
            except Exception as e:
                print(f"Error processing {surface_file}: {e}")
        else:
            print(f"Surface file not found: {surface_file}")

        # Process pressure file
        if os.path.exists(pressure_file):
            try:
                pf = pq.ParquetFile(pressure_file)
                file_columns = set(pf.schema.names)
                cols_to_read = list(set(pressure_columns).intersection(file_columns))
                if not cols_to_read:
                    print(f"No matching pressure columns in {pressure_file}")
                else:
                    for batch in pf.iter_batches(columns=cols_to_read):
                        df_chunk = batch.to_pandas()
                        update_aggregates(df_chunk, pressure_columns, pressure_sum, pressure_count)
                        if "u_mean" in df_chunk.columns and "v_mean" in df_chunk.columns:
                            speeds = np.sqrt(df_chunk["u_mean"]**2 + df_chunk["v_mean"]**2)
                            pressure_wind_speed_sum += speeds.sum()
                            pressure_wind_speed_count += speeds.count()
                            theta = np.degrees(np.arctan2(df_chunk["v_mean"], df_chunk["u_mean"]))
                            directions = (270.0 - theta) % 360.0
                            rad = np.radians(directions)
                            pressure_wind_dir_sin_sum += np.sin(rad).sum()
                            pressure_wind_dir_cos_sum += np.cos(rad).sum()
                            pressure_wind_dir_count += len(rad)
            except Exception as e:
                print(f"Error processing {pressure_file}: {e}")
        else:
            print(f"Pressure file not found: {pressure_file}")

# ------------------------- COMPUTE GLOBAL MEANS -------------------------
def compute_means(sum_dict, count_dict):
    means = {}
    for col, total in sum_dict.items():
        cnt = count_dict.get(col, 0)
        means[col] = total / cnt if cnt > 0 else None
    return means

surface_means = compute_means(surface_sum, surface_count)
pressure_means = compute_means(pressure_sum, pressure_count)

# ------------------------- TEMPERATURE CONVERSION -------------------------
for tvar in surface_temp_vars:
    if tvar in surface_means and surface_means[tvar] is not None:
        surface_means[tvar] -= 273.15
        param_units[tvar] = "°C"
for tvar in pressure_temp_vars:
    if tvar in pressure_means and pressure_means[tvar] is not None:
        pressure_means[tvar] -= 273.15
        param_units[tvar] = "°C"

# ------------------------- WIND CALCULATIONS -------------------------
# For surface wind: compute average wind speed and circular mean wind direction
if surface_wind_speed_count > 0:
    avg_surface_wind_speed = surface_wind_speed_sum / surface_wind_speed_count
    avg_sin = surface_wind_dir_sin_sum / surface_wind_dir_count
    avg_cos = surface_wind_dir_cos_sum / surface_wind_dir_count
    avg_surface_wind_dir = math.degrees(math.atan2(avg_sin, avg_cos)) % 360
    surface_means["wind_speed_10m"] = avg_surface_wind_speed
    surface_means["wind_dir_10m"] = avg_surface_wind_dir
    param_units["wind_speed_10m"] = "m/s"
    param_units["wind_dir_10m"] = "deg_from_N"
else:
    print("No surface wind observations to compute wind speed/direction.")

# For pressure wind:
if pressure_wind_speed_count > 0:
    avg_pressure_wind_speed = pressure_wind_speed_sum / pressure_wind_speed_count
    avg_sin = pressure_wind_dir_sin_sum / pressure_wind_dir_count
    avg_cos = pressure_wind_dir_cos_sum / pressure_wind_dir_count
    avg_pressure_wind_dir = math.degrees(math.atan2(avg_sin, avg_cos)) % 360
    pressure_means["wind_speed_pressure"] = avg_pressure_wind_speed
    pressure_means["wind_dir_pressure"] = avg_pressure_wind_dir
    param_units["wind_speed_pressure"] = "m/s"
    param_units["wind_dir_pressure"] = "deg_from_N"
else:
    print("No pressure wind observations to compute wind speed/direction.")

# ------------------------- SAVE RESULTS -------------------------
def save_results(means_dict, filename):
    results = []
    for param, mean_val in means_dict.items():
        unit = param_units.get(param, "")
        results.append({"Parameter": param, "Mean_Value": mean_val, "Units": unit})
    df = pd.DataFrame(results)
    out_path = os.path.join(output_dir, filename)
    df.to_csv(out_path, index=False)
    print(f"Results saved to {out_path}")

save_results(surface_means, "ERA5_surface_means_with_wind_calcs_chunked.csv")
save_results(pressure_means, "ERA5_pressure_means_with_wind_calcs_chunked.csv")
